In [24]:
import psycopg2
import pandas as pd
import psycopg2.extras as extras
import os
from os import listdir
from os.path import isfile, join

In [25]:
os.chdir("C:/Users/pauli/Documents/M2/Text Mining/projet/Disney-Text-Mining/fichiers/tables")
mypath = os.getcwd()


client = pd.read_csv(str(mypath)+"/client.csv",sep=';')
date = pd.read_csv(str(mypath)+"/date.csv",sep=';')
hotel = pd.read_csv(str(mypath)+"/hotel.csv",sep=';')
room = pd.read_csv(str(mypath)+"/room.csv",sep=';')
reservation = pd.read_csv(str(mypath)+"/reservation.csv",sep=';')

# Oracle

In [26]:
#??
#compass club 
#empire state club 

# MySQL

In [27]:
#??

# PostgreSQL

In [28]:
#https://waytolearnx.com/2020/06/connexion-a-une-base-de-donnees-postgresql-en-python.html

In [29]:
conn = psycopg2.connect(
          user = "m139",
          password = "m139",
          host = "db-etu.univ-lyon2.fr",
          port = "5432",
          database = "m139"
    )

# execute request 

In [30]:
def execute_req(conn, req):
    try: 
        cursor = conn.cursor()
        cursor.execute(req)
        conn.commit()
        cursor.close()
    except (Exception, psycopg2.Error) as error :
        print ("Erreur lors de la création du table PostgreSQL", error)

# drop

In [31]:
sql_drop_reservation = '''DROP TABLE reservation; '''
execute_req(conn, sql_drop_reservation)

In [32]:
sql_drop_date = '''DROP TABLE date; '''
execute_req(conn, sql_drop_date)

In [33]:
sql_drop_client = '''DROP TABLE client; '''
execute_req(conn, sql_drop_client)

In [34]:
sql_drop_room = '''DROP TABLE room; '''
execute_req(conn, sql_drop_room)

In [35]:
sql_drop_hotel = '''DROP TABLE hotel; '''
execute_req(conn, sql_drop_hotel)

## create tables

In [36]:
sql_create_hotel = '''CREATE TABLE HOTEL(
    id_hotel TEXT PRIMARY KEY NOT NULL,
    hotel VARCHAR(100),
    level_hotel INT
  ); '''

execute_req(conn, sql_create_hotel)

In [37]:
sql_create_room = '''CREATE TABLE ROOM(
    id_room TEXT PRIMARY KEY NOT NULL,
    room_type TEXT,
    id_hotel TEXT,
    CONSTRAINT ROOM_ibfk_1 FOREIGN KEY (id_hotel) REFERENCES HOTEL(id_hotel)
  ); '''
execute_req(conn, sql_create_room)

In [38]:
sql_create_client = '''CREATE TABLE CLIENT(
    "id_client" TEXT PRIMARY KEY NOT NULL,
    country TEXT,
    nuitee INT,
    traveler_infos TEXT,
    review_title TEXT, 
    positive_review TEXT,
    negative_review TEXT, 
    usefulness_review TEXT,
    delay_comment INT
  ); '''
execute_req(conn, sql_create_client)


In [39]:
sql_create_date = '''CREATE TABLE DATE(
    id_date TEXT PRIMARY KEY NOT NULL,
    month_str TEXT,
    year INT,
    month_num INT,
    date DATE
  ); '''
execute_req(conn, sql_create_date)


In [40]:
sql_create_res = '''CREATE TABLE RESERVATION(
    id_reservation TEXT PRIMARY KEY NOT NULL,
    grade_review FLOAT,
    level_grade_review INT,
    id_room TEXT,
    id_date TEXT,
    id_client TEXT,
    CONSTRAINT RESERVATION_ibfk_1 FOREIGN KEY (id_room) REFERENCES ROOM(id_room),
    CONSTRAINT RESERVATION_ibfk_2 FOREIGN KEY (id_date) REFERENCES DATE(id_date),
    CONSTRAINT RESERVATION_ibfk_3 FOREIGN KEY (id_client) REFERENCES CLIENT(id_client)
  ); '''
execute_req(conn, sql_create_res)
    

# insert values

In [41]:
def insert_values(conn, df, table):
    tuples = [tuple(x) for x in df.to_numpy()]
    cols = ','.join(list(df.columns))
    # SQL query to execute
    query = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
    cursor = conn.cursor()
    try:
        extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("the dataframe is inserted")
    cursor.close()

In [42]:
insert_values(conn, date, 'date')

the dataframe is inserted


In [43]:
insert_values(conn, hotel, 'hotel')

the dataframe is inserted


In [44]:
insert_values(conn, room, 'room')

the dataframe is inserted


In [45]:
insert_values(conn, client, 'client')

the dataframe is inserted


In [46]:
insert_values(conn, reservation, 'reservation')

the dataframe is inserted


## get tables

In [52]:
try:
    conn = psycopg2.connect(
          user = "m139",
          password = "m139",
          host = "db-etu.univ-lyon2.fr",
          port = "5432",
          database = "m139"
    )
    cur = conn.cursor()
    reservation = "SELECT * FROM reservation"
    client = "SELECT * FROM client"
    hotel = "SELECT * FROM hotel"
    room = "SELECT * FROM room"
    date = "SELECT * FROM date"

    cur.execute(reservation)
    reservation = pd.DataFrame(cur.fetchall(), columns=[desc[0] for desc in cur.description])
    cur.execute(client)
    client = pd.DataFrame(cur.fetchall(), columns=[desc[0] for desc in cur.description])
    cur.execute(hotel)
    hotel = pd.DataFrame(cur.fetchall(), columns=[desc[0] for desc in cur.description])
    cur.execute(room)
    room = pd.DataFrame(cur.fetchall(), columns=[desc[0] for desc in cur.description])
    cur.execute(date)
    date = pd.DataFrame(cur.fetchall(), columns=[desc[0] for desc in cur.description])

    cur.close()
    conn.close()
except (Exception, psycopg2.Error) as error :
    print ("Erreur lors de la connexion à PostgreSQL", error)


hotel_room = hotel.merge(room, on="id_hotel")
res_client = reservation.merge(client, on="id_client")
res_client_date = res_client.merge(date, on="id_date")
df = res_client_date.merge(hotel_room, on="id_room")
df

,id_reservation,grade_review,level_grade_review,id_room,id_date,id_client,country,nb_night,traveler_infos,review_title,positive_review,negative_review,usefulness_review,month_str,year,month_num,date,id_hotel,hotel,room_type
0,403ca88b-ea0d-4382-bf18-b77776e9a96a,9.0,2,b9203b91-368b-4899-8427-12a1b66183e4,20230101,08f81258-3232-4a0b-b520-180e44269abf,Royaume-Uni,1,Famille,excellent and memorable,NaN,NaN,0,janvier,2023,1,2023-01-01,d1c15602-b86b-4eff-bfb7-46bb7dd0dc1d,Cheyenne,Chambre Standard Woody's Roundup
1,d20f9677-0403-4d35-9734-735dcd5bf9b9,7.0,1,b9203b91-368b-4899-8427-12a1b66183e4,20230101,1a598c5b-8687-4f31-834e-b2b17ce2ae64,Belgique,1,Famille,Bien,NaN,NaN,0,janvier,2023,1,2023-01-01,d1c15602-b86b-4eff-bfb7-46bb7dd0dc1d,Cheyenne,Chambre Standard Woody's Roundup
2,491bfe50-de22-4163-b0bf-c0a1b449db13,10.0,2,b9203b91-368b-4899-8427-12a1b66183e4,20230101,18b44676-bcbc-4c5f-862b-02f77b934d1c,Royaume-Uni,2,Famille,amazing and very helpful staff,NaN,NaN,0,janvier,2023,1,2023-01-01,d1c15602-b86b-4eff-bfb7-46bb7dd0dc1d,Cheyenne,Chambre Standard Woody's Roundup
3,fcd6725a-2494-4f97-96a9-27d9dabedb65,9.0,2,b9203b91-368b-4899-8427-12a1b66183e4,20230101,caa9a5d7-23cf-4804-a6ef-48079a51429e,France,1,Famille,On a passé un très bon moment !!,proximité du parc. décoration. propreté de la ...,la ventilation du chauffage qui faisait du bruit.,0,janvier,2023,1,2023-01-01,d1c15602-b86b-4eff-bfb7-46bb7dd0dc1d,Cheyenne,Chambre Standard Woody's Roundup
4,b2a8c4ed-34d0-46ab-ba03-bce7a4404abe,8.0,2,b9203b91-368b-4899-8427-12a1b66183e4,20230101,e2310d14-5127-4959-934f-36e9d8a71841,Pays-Bas,1,Famille,"Fijn verblijf gehad, hebben nacht bijgeboekt e...",NaN,NaN,0,janvier,2023,1,2023-01-01,d1c15602-b86b-4eff-bfb7-46bb7dd0dc1d,Cheyenne,Chambre Standard Woody's Roundup
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11214,45a996b4-0a93-4846-94c1-f3972412500a,8.0,2,b3d1970c-2de1-45d7-a7e6-dc9146172945,20191201,7e71f00d-8fcc-4b15-88a4-ed785e982f16,Pays-Bas,1,Groupe,Très bien,NaN,NaN,0,décembre,2019,12,2019-12-01,806fd917-24c9-44b2-990b-6973cee9c360,Davy_Crockett_Ranch,Bungalow Tribu Pionnier (5 Adultes + 1 Enfant)
11215,ddf84ad0-cb9b-4157-8950-1b61b8b38dd8,9.6,2,ea2fec9b-84d1-436f-a90e-637e15d1b71d,20200101,7227cb88-462e-4f9e-8401-367183a77e60,La Réunion,1,Famille,Exceptionnel,NaN,NaN,0,janvier,2020,1,2020-01-01,d00da4f6-b100-4cc5-bac5-c8508c4bc35a,Newport_Bay_Club,Chambre Club Compass (3 Adultes + 1 Enfant)
11216,7d72609b-938f-4ba3-bdeb-8d71a1b4722d,10.0,2,ea2fec9b-84d1-436f-a90e-637e15d1b71d,20200301,e5b1e82c-1794-4ef3-a423-63748585dca2,France,3,Famille,Exceptionnel,Hôtel charmant très bon accueil,NaN,0,mars,2020,3,2020-03-01,d00da4f6-b100-4cc5-bac5-c8508c4bc35a,Newport_Bay_Club,Chambre Club Compass (3 Adultes + 1 Enfant)
11217,09635a1a-fcba-4d79-95d1-19c4b969c201,8.0,2,8bbaa52a-ca7a-4de9-a02a-83100de9d83a,20200301,de6cf0d8-2378-498f-b82d-141d39e7a6bf,Belgique,2,Voyageur individuel,Très bien,Hôtel tres agréable et les employés sont très ...,Literie qui n'est pas au top,1,mars,2020,3,2020-03-01,ab287e48-2017-4844-bbf2-d304cdc28d90,Sequoia_Lodge,Chambre Standard proche Installations Hôtelièr...
